In [43]:
import pandas as pd
import numpy as np

In [2]:
# Load data
df = pd.read_csv("data/ufo_sighting_data.csv")
df.shape

C:\Users\jonmw\AppData\Local\Temp\ipykernel_30612\2855359057.py:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/ufo_sighting_data.csv")


(80332, 11)

In [3]:
from prettytable import PrettyTable

def pretty_table_missing_counts(df, cardinality_limit=10, display=True):
  '''
  Method takes 3 parameters (1 required and 2 optional) and creates a visually pleasing table using the PrettyTable package
  that displays all columns, datatypes, non-null/null counts, and whether cardinality is high
  Params:
    ► df (DataFrame) | Pandas DataFrame
    ► cardinality_limit (int) | Integer representing the limit for considering a column to have high cardinality. Default = 10 items
    ► display (bool) | True/False whether to print the table before returning. Default = True
  Return:
    ► PrettyTable table
  '''

  table = PrettyTable()

  table.field_names = [
      'Column Name', 'Data Type', 'Non-Null Count', 'Missing Count', 'Unique', 
      'High Cardinality'
  ]

  for column in df.columns:
      data_type = str(df[column].dtype)
      non_null_count = df[column].count()
      missing_count = df.shape[0] - non_null_count
      uniques = df[column].nunique()
      cardinality = uniques > cardinality_limit
      table.add_row([column, data_type, non_null_count, missing_count, uniques, cardinality])
  
  if display:
    print(table)
    
  return table

a = pretty_table_missing_counts(df, display=True)

+---------------------------------+-----------+----------------+---------------+--------+------------------+
|           Column Name           | Data Type | Non-Null Count | Missing Count | Unique | High Cardinality |
+---------------------------------+-----------+----------------+---------------+--------+------------------+
|            Date_time            |   object  |     80332      |       0       | 69586  |       True       |
|               city              |   object  |     80332      |       0       | 19900  |       True       |
|          state/province         |   object  |     74535      |      5797     |   67   |       True       |
|             country             |   object  |     70662      |      9670     |   5    |      False       |
|            UFO_shape            |   object  |     78400      |      1932     |   29   |       True       |
|   length_of_encounter_seconds   |   object  |     80332      |       0       |  705   |       True       |
| described_duratio

In [4]:
df = df.drop(df[pd.to_numeric(df['latitude'], errors='coerce').isna()].index).reset_index(drop=True)
df = df.drop(df[pd.to_numeric(df['longitude'], errors='coerce').isna()].index).reset_index(drop=True)

In [5]:
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [6]:
df = df.drop(df[pd.to_numeric(df['length_of_encounter_seconds'], errors='coerce').isna()].index).reset_index(drop=True)
df['length_of_encounter_seconds'] = df['length_of_encounter_seconds'].astype(float)

In [8]:
# Replace "24:00" with "00:00"
df['Date_time'] = df['Date_time'].str.replace('24:00', '00:00')
df['Date_time'] = pd.to_datetime(df['Date_time'], format='%m/%d/%Y %H:%M')

In [19]:
df['Years'] = df['Date_time'].dt.year
df['Months'] = df['Date_time'].dt.month
df['Hour'] = df['Date_time'].dt.hour

In [17]:
# Define a custom mapping function to categorize months into seasons
def map_to_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

# Apply the mapping function to create a 'Season' column
df['Season'] = df['Months'].apply(map_to_season)

In [28]:
df['length_of_encounter_seconds'].value_counts(sort=True)

length_of_encounter_seconds
300.00     8635
120.00     7110
600.00     6312
60.00      5866
180.00     5064
           ... 
199.00        1
1281.00       1
1420.00       1
120.15        1
181.00        1
Name: count, Length: 533, dtype: int64

In [22]:
df

,Date_time,city,state/province,country,UFO_shape,length_of_encounter_seconds,described_duration_of_encounter,description,date_documented,latitude,longitude,Years,Months,Season,Hour
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,1949,10,Autumn,20
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,1949,10,Autumn,21
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,1955,10,Autumn,17
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,1956,10,Autumn,21
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,1960,10,Autumn,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80323,2013-09-09 21:15:00,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444,2013,9,Autumn,21
80324,2013-09-09 22:00:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500,2013,9,Autumn,22
80325,2013-09-09 22:00:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444,2013,9,Autumn,22
80326,2013-09-09 22:20:00,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556,2013,9,Autumn,22


In [30]:
# Define a custom lambda function to join string columns into a multi-line string with custom formatting
def custom_format(row):
    entries = []
    for column_name, value in row.items():
        entries.append(f'{column_name}: {value}')
    return '\n'.join(entries)


df['Text'] = df[['Years', 'Season', 'UFO_shape', 'described_duration_of_encounter']].apply(custom_format, axis=1)
print(df)

                Date_time                  city state/province country  \
0     1949-10-10 20:30:00            san marcos             tx      us   
1     1949-10-10 21:00:00          lackland afb             tx     NaN   
2     1955-10-10 17:00:00  chester (uk/england)            NaN      gb   
3     1956-10-10 21:00:00                  edna             tx      us   
4     1960-10-10 20:00:00               kaneohe             hi      us   
...                   ...                   ...            ...     ...   
80323 2013-09-09 21:15:00             nashville             tn      us   
80324 2013-09-09 22:00:00                 boise             id      us   
80325 2013-09-09 22:00:00                  napa             ca      us   
80326 2013-09-09 22:20:00                vienna             va      us   
80327 2013-09-09 23:00:00                edmond             ok      us   

      UFO_shape  length_of_encounter_seconds described_duration_of_encounter  \
0      cylinder                

In [49]:
sorted([shape.capitalize() for shape in list(df['UFO_shape'].unique()) if shape is not np.nan])

['Changed',
 'Changing',
 'Chevron',
 'Cigar',
 'Circle',
 'Cone',
 'Crescent',
 'Cross',
 'Cylinder',
 'Delta',
 'Diamond',
 'Disk',
 'Dome',
 'Egg',
 'Fireball',
 'Flare',
 'Flash',
 'Formation',
 'Hexagon',
 'Light',
 'Other',
 'Oval',
 'Pyramid',
 'Rectangle',
 'Round',
 'Sphere',
 'Teardrop',
 'Triangle',
 'Unknown']